# Reverse Game of Life - Inception CNN

In [ ]:
# Download git repository and copy to local directory
!rm -rf /ai-games/
!git clone https://github.com/JamesMcGuigan/ai-games/ /ai-games/
# !cd /ai-games/; git checkout ad2f8cc94865f1be6083ca699d4b62b0cc039435
!cp -rf /ai-games/puzzles/game_of_life/* ./   # copy code to kaggle notebook
!cd /ai-games/; git log -n1 

In [ ]:
from numba import njit, prange
from scipy.signal import convolve2d
from typing import Union, List, Tuple, Dict, Callable
from itertools import chain, product

import humanize
import itertools
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy
import scipy.sparse
import sys
import time
import skimage
import skimage.measure
import pydash

import torch
import numpy as np
from utils.util import *
from utils.plot import *
from utils.game import *
from utils.datasets import *
from utils.tuplize import *
from hashmaps.crop import *
from hashmaps.hash_functions import *
from hashmaps.translation_solver import *
from hashmaps.repeating_patterns import *
from constraint_satisfaction.fix_submission import *
from neural_networks.train import train



device   = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
__file__ = './notebook.py'
notebook_start = time.perf_counter()

### Don't wrap console output text
from IPython.display import display, HTML
display(HTML("""<style>
div.output_area pre {
    width: 10000px;
}
</style>"""))

In [ ]:
from abc import ABCMeta
from typing import TypeVar

import numpy as np
import torch
import torch as pt
import torch.nn.functional as F
from torch import nn

from neural_networks.GameOfLifeBase import GameOfLifeBase
from neural_networks.hardcoded.GameOfLifeHardcodedReLU1_21 import GameOfLifeHardcodedReLU1_21
from neural_networks.modules.ReLUX import ReLU1

# noinspection PyTypeChecker
T = TypeVar('T', bound='GameOfLifeFeatures')
class GameOfLifeFeatures(nn.Module):
    """ Count the number of neighbouring cells (excluding self)"""
    def __init__(self):
        super().__init__()
        self.forward_play = GameOfLifeHardcodedReLU1_21()
        self.conv2d       = nn.Conv2d(in_channels=1, out_channels=4, bias=False,
                                      kernel_size=(3,3), padding=1, padding_mode='circular')
        self.conv2d.weight.data = torch.tensor([
            # 11N Solution
            [[[ -1.0, -1.0, -1.0 ],
              [ -1.0, -0.7, -1.0 ],
              [ -1.0, -1.0, -1.0 ]]],
            # neighbours
            [[[  1.0,  1.0,  1.0 ],
              [  1.0,  0.0,  1.0 ],
              [  1.0,  1.0,  1.0 ]]],
            # # corners
            # [[[  1.0,  0.0,  1.0 ],
            #   [  0.0,  0.0,  0.0 ],
            #   [  1.0,  0.0,  1.0 ]]],
            # # edges
            # [[[  0.0,  1.0,  0.0 ],
            #   [  1.0,  0.0,  1.0 ],
            #   [  0.0,  1.0,  0.0 ]]],
        ])
        self.conv2d.weight.requires_grad_(False)  # Weights are hardcoded


    def forward(self, x):
        shape = x.shape
        x = x.reshape(-1, 1, x.shape[2], x.shape[3])
        features = self.conv2d(x)
        x = torch.cat([
            x,                     # +1 channels
            self.forward_play(x),  # +1 channels
            features,              # +4 channels
        ], dim=1)
        x = x.reshape(shape[0], -1, shape[2], shape[3])
        return x




class GameOfLifeReverseInception(GameOfLifeBase, metaclass=ABCMeta):
    """
    """
    def __init__(self):
        super().__init__()

        # self.criterion = FocalLoss()
        self.criterion = nn.MSELoss()
        # self.criterion  = nn.BCELoss()
        self.activation = nn.PReLU()
        self.relu1      = ReLU1()

        # discriminator must be at top-level for autograd to work, its weights are added to the savefile
        self.discriminator = GameOfLifeHardcodedReLU1_21()
        for name, parameter in self.discriminator.named_parameters(): parameter.requires_grad = False

        self.features = GameOfLifeFeatures()

        self.layers = nn.ModuleList([

            # self.features()
            nn.ModuleList([
                nn.Conv2d(in_channels=4, out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),
                nn.Conv2d(in_channels=4, out_channels=9, kernel_size=(3,3), padding=1, padding_mode='circular'),
                nn.Conv2d(in_channels=4, out_channels=9, kernel_size=(5,5), padding=2, padding_mode='circular'),
                nn.Conv2d(in_channels=4, out_channels=9, kernel_size=(7,7), padding=3, padding_mode='circular'),
                nn.Conv2d(in_channels=4, out_channels=9, kernel_size=(9,9), padding=4, padding_mode='circular'),
            ]),
            nn.Conv2d(in_channels=9*5, out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),
            nn.Conv2d(in_channels=9,   out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),

            # self.features()
            nn.ModuleList([
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(3,3), padding=1, padding_mode='circular'),
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(5,5), padding=2, padding_mode='circular'),
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(7,7), padding=3, padding_mode='circular'),
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(9,9), padding=4, padding_mode='circular'),
            ]),
            nn.Conv2d(in_channels=9*5, out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),
            nn.Conv2d(in_channels=9,   out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),

            # self.features()
            nn.ModuleList([
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(3,3), padding=1, padding_mode='circular'),
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(5,5), padding=2, padding_mode='circular'),
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(7,7), padding=3, padding_mode='circular'),
                nn.Conv2d(in_channels=(9+1)*4, out_channels=9, kernel_size=(9,9), padding=4, padding_mode='circular'),
            ]),
            nn.Conv2d(in_channels=9*5, out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),
            nn.Conv2d(in_channels=9,   out_channels=9, kernel_size=(1,1), padding=0, padding_mode='circular'),

            nn.Conv2d(in_channels=9,   out_channels=1, kernel_size=(1,1), padding=0, padding_mode='circular'),
        ])


    def forward(self, x):
        x = self.cast_inputs(x)
        x = self.relu1(x)
        input = self.features(x)
        for n, layer in enumerate(self.layers):
            if isinstance(layer, nn.ModuleList):
                x = self.features(x)
                x = torch.cat([ x, input ], dim=1) if n != 0 else x
                channels = [ sublayer(x) for sublayer in layer ]
                x = torch.cat(channels, dim=1)
            else:
                x = layer(x)
            x = self.activation(x)
        x = self.relu1(x)
        return x


    def loss(self, outputs: pt.tensor, expected: pt.tensor, inputs: pt.tensor):
        """
        GameOfLifeReverseOneGAN() computes the backwards timestep
        discriminator GameOfLifeHardcodedReLU1_21() replays the board again forwards
        forward_loss is the MSE difference between the backwards prediction and forward play
        classic_loss biases the network towards the exact solution, but reduces to zero as forward_loss approaches zero
        cell_count_loss is a heuristic to guide solution towards the correct cell count and avoid all 0 or all 1 solutions
        binary_loss penalizes non-binary output
        """
        forwards         = self.discriminator(outputs)
        forward_loss     = self.criterion(forwards, inputs)                      # loss==0 if forward play matches input
        classic_loss     = self.criterion(outputs, expected)                     # loss==0 if dataset matches output
        cell_count_loss1 = ( torch.mean(forwards) - torch.mean(inputs)   ) ** 2  # loss==0 if cell count is the same
        cell_count_loss2 = ( torch.mean(outputs)  - torch.mean(expected) ) ** 2  # loss==0 if cell count is the same
        binary_loss1     = torch.mean( 0.5**2 - ( forwards - 0.5 )**2 )          # loss==0 if all values are 1 or 0
        binary_loss2     = torch.mean( 0.5**2 - ( outputs  - 0.5 )**2 )          # loss==0 if all values are 1 or 0
        loss = (
            forward_loss + cell_count_loss1 + binary_loss1 + binary_loss2
            + F.relu( ( classic_loss + cell_count_loss2 ) * (forward_loss - 0.01) * 10 )
        )
        return loss

    # noinspection PyTypeChecker
    def accuracy(self, outputs, expected, inputs):
        """ Accuracy here is based upon if the output matches the input after forward play """
        # return super().accuracy(outputs, expected, inputs)
        forwards = self.discriminator(self.cast_int(outputs))
        return pt.sum( self.cast_bool(forwards) == self.cast_bool(inputs) ).cpu().numpy() / np.prod(outputs.shape)


    def unfreeze(self: T) -> T:
        if not self.loaded: self.load()
        excluded = { 'discriminator', 'features' }
        for name, parameter in self.named_parameters():
            if not set( name.split('.') ) & excluded:
                parameter.requires_grad = True
        return self
    
    def weights_init(self, layer):
        pass

In [ ]:
# %%time
# from neural_networks.train import train

# # Keep trying to retrain the network until we get 90% accuracy after 300 epochs
# def lottery_ticket_initialization( model_class, epochs=100, min_acc=0.9, timeout=30*60, verbose=False ):
#     time_start = time.perf_counter()
#     accs     = []
#     expected = generate_random_boards(1_000)
#     inputs   = [ life_step(board) for board in expected ]
#     best_model = None
#     best_acc   = -1
#     while True:
#         model = model_class()
#         if os.path.exists(model.filename): os.remove(model.filename)
#         model.load(verbose=verbose)
        
#         # small grids have less whitespace
#         train(model, grid_size=5, reverse_input_output=True, epochs=epochs, verbose=verbose)  
        
#         acc = np.mean( model.predict(inputs) == expected ) 
#         accs.append( acc )
#         if acc > best_acc: 
#             best_model = model
#             best_acc   = acc
#         if acc > min_acc:  
#             model.save() 
#             break
#         if timeout and timeout < time.perf_counter() - time_start: 
#             break
#     print(f'attempts: {len(accs)} | accuracies: {accs} | min: {np.min(accs)} | mean: {np.mean(accs)} | max: {np.max(accs)} | std: {np.std(accs)}')
#     return best_model

# model = lottery_ticket_initialization(GameOfLifeReverseInception)

model = GameOfLifeReverseInception()

In [ ]:
timeout = 110 * 60 - ( time.perf_counter() - notebook_start )
train(model, grid_size=3, reverse_input_output=True, timeout=timeout)

In [ ]:
timeout = 110 * 60 - ( time.perf_counter() - notebook_start )
train(model, grid_size=5, reverse_input_output=True, timeout=timeout)

In [ ]:
timeout = 110 * 60 - ( time.perf_counter() - notebook_start )
train(model, grid_size=7, reverse_input_output=True, timeout=timeout)

In [ ]:
timeout = 110 * 60 - ( time.perf_counter() - notebook_start )
train(model, grid_size=11, reverse_input_output=True, timeout=timeout)

In [ ]:
timeout = 110 * 60 - ( time.perf_counter() - notebook_start )
train(model, grid_size=25, reverse_input_output=True, timeout=timeout)